In [2]:
from bs4 import BeautifulSoup
import requests
import sqlite3 as sql
import pandas as pd
import duckdb as ddb

conn = sql.connect('Celebrity.db')

# Main Scraping of Data

In [16]:
# Add a List of Organizations
Organizations = ["peta".lower()]

for Organization in Organizations:
  url = f'https://www.looktothestars.org/charity/{Organization}'
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  supporters = soup.find('section', {'id': 'supporters'})
  celebrities = supporters.find_all('div', {'class': 'media-body'})

  List_Celebrities = []
  for celebrity in celebrities:
    name_clean = celebrity.text
    combined_name = [Organization, name_clean]
    List_Celebrities.append(combined_name)

CelebrityOrg_df = pd.DataFrame(List_Celebrities, columns=['Organization', 'Name'])
CelebrityOrg_df = CelebrityOrg_df.drop_duplicates(subset=['Name']).copy()


## Pulling the Kaggle Dataset that includes the occupation and Yearly income 

### How to Use .drop_duplicates
- df.drop_duplicates(subset=['column_name1', 'column_name2'])
    - Looks at just the columns specified
- df.drop_duplicates(keep='  ')
    - "Last" = Keeps Last Occurence
    - None = Removes all occurences that are duplicates

### Process Kaggle Dataset and Combine

In [4]:
kaggle_celebrities = pd.read_excel("list of celebrities from Kaggle.xlsx")
Unique_Kaggle_Celebrities = kaggle_celebrities.drop_duplicates(subset=['Name']).copy()
Unique_Kaggle_Celebrities['Pay'] = Unique_Kaggle_Celebrities['Pay (USD millions)']
Kaggle_Celebrities_df = Unique_Kaggle_Celebrities.drop(['Year','Pay (USD millions)'], axis= 1)

# Combining Kaggle with WebScraped
qry_combine = '''
    SELECT CO.Name, CO.Organization, CK.Category, CK.Pay
    FROM CelebrityOrg_df AS CO
    JOIN Kaggle_Celebrities_df AS CK ON CO.Name = CK.Name
'''

Names_on_Kaggle = ddb.sql(qry_combine).df()
print(Names_on_Kaggle.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          15 non-null     object 
 1   Organization  15 non-null     object 
 2   Category      15 non-null     object 
 3   Pay           15 non-null     float64
dtypes: float64(1), object(3)
memory usage: 612.0+ bytes
None


### Update the Webscraped DB marking the Kaggle Matches

In [24]:

merged_df = pd.merge(CelebrityOrg_df, Names_on_Kaggle, on='Name', how='left')
Org_Celeb_merged_df = merged_df.drop(['Organization_y'],axis=1)

Organization_df = []
celebrity_df = []

# Adding the Celeb and Org IDs now because it is easier
CelebID_df = Org_Celeb_merged_df[['Name']].drop_duplicates(subset=['Name']).copy()
CelebID_df['Celeb_ID'] = range(1, len(CelebID_df)+1)
OrgID_df = Org_Celeb_merged_df[['Organization_x']].drop_duplicates(subset=['Organization_x']).copy()
OrgID_df['Org_ID'] = range(1000, 1000 + len(OrgID_df))

Org_Celeb_merged_df = pd.merge(Org_Celeb_merged_df, CelebID_df, on='Name', how='inner')
Org_Celeb_merged_df = pd.merge(Org_Celeb_merged_df, OrgID_df, on='Organization_x', how='inner')

Org_Celeb_merged_df.to_sql('Combined_Org_Celeb', conn, if_exists='replace', index=False)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 268
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Organization_x  269 non-null    object 
 1   Name            269 non-null    object 
 2   Category        15 non-null     object 
 3   Pay             15 non-null     float64
 4   Celeb_ID        269 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 12.6+ KB
None
  Organization_x              Name Category  Pay  Celeb_ID  Org_ID
0           peta  Adriana de Moura      NaN  NaN         1    1000
1           peta   Aimee Teegarden      NaN  NaN         2    1000
2           peta     Aishwarya Rai      NaN  NaN         3    1000
3           peta      Alan Cumming      NaN  NaN         4    1000
4           peta       Alan Davies      NaN  NaN         5    1000


269

# Creating RDB
### Structure
- Celebrity_tbl
    - Celeb_ID
    - Full Name
    - Category 
    - Pay
    - Accounted_For (T/F)

- Organization_tbl
    - Org_ID
    - Description (Needs to be added)




In [25]:
# Normalizing the Data with Multiple Tables

cursor = conn.cursor()

celeb_qry = '''
    SELECT Celeb_ID, Name, FIRST(Category) AS Category, FIRST(Pay) AS Salary
    FROM Combined_Org_Celeb
    GROUP BY Name, Celeb_ID 
'''
Org_qry = '''
    SELECT Org_ID, Organization_x AS Organization
    FROM Combined_Org_Celeb
    GROUP BY Org_ID, Organization
'''
# Because it will be a Many to Many Relationship
Join_tbl_query = '''
    SELECT Celeb_ID, Org_ID
    FROM Combined_Org_Celeb
    GROUP BY Org_ID, Celeb_ID
'''

create_celeb_tbl = f"CREATE TABLE Celeberties AS {celeb_qry}"
create_org_tbl = f"CREATE TABLE Chartered_Orgs AS {Org_qry}"
create_Join_tbl = f"CREATE TABLE Join_tbl AS {Join_tbl_query}"
cursor.execute(create_celeb_tbl)
cursor.execute(create_org_tbl)
cursor.execute(create_Join_tbl)
conn.commit()

cursor.close()
conn.close()


OperationalError: no such function: FIRST

# Testing

In [ ]:
cursor = conn.cursor()

table_name ="Results_Combined_Celebrities"
query = '''
SELECT CO.Name, CO.Organization, CK.Category, CK.Pay
FROM Celebrity_Org AS CO
JOIN Kaggle_Celebrities AS CK ON CO.Name = CK.Name
'''

df = pd.read_sql_query(query, conn)
print(df.info())

'''
create_table_qry = f"CREATE TABLE {table_name} AS {query}"
cursor.execute(create_table_qry)
conn.commit()
conn.close()
'''
cursor.close()
